In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
Using cached langchain_experimental-0.3.4-py3-none-any.whl (209 kB)
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.0 MB 4.2 MB/s eta 0:00:01
   ------------------ --------------------- 1.8/4.0 MB 5.3 MB/s eta 0:00:01
   ---------------------------- ----------- 2.9/4.0 MB 4.9 MB/s eta 0:00:01
   ------------------------------------ --- 3.7/4.0 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   --------- ------------------------------ 0.8/3.1 MB 3.4 MB/s eta 0:00:01
   ------------------- -------------------- 1.6/3.1 MB 3.6 MB/s eta 0:00:01
   ----------------------------- ---------- 2.4/3.1 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 3.1/3.1 MB 3.7 MB/s eta 0:00:00
  Attempting uninstall: lxml
  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.31.1 which is incompatible.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


## Built-in Tool - DuckDuckGo Search

In [6]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

C:\Users\heman\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


1 day ago · Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. 1 day ago · Top News Stories of the day, Latest News Headlines, News Specials, Breaking News and Latest India News, World current affairs & Political News all around the clock at NDTV.com. Feb 28, 2024 · India News Today: Get latest India news and breaking news from India on Times of India. TOI brings the top news headlines from India on Politics, Elections, Current Affairs … 1 day ago · Stay Up-to-date with Top news in India, current headlines, photos & videos online, live coverage on entertainment, business, politics, sports, technology and more at … 1 day ago · Latest news headlines from India & around the world. Check out today’s news coverage live with videos & photos on NDTV.com.


In [3]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

In [7]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('whoami')

print(results)

Executing command:
 whoami
hemant\heman



C:\Users\heman\anaconda3\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom Tools

In [8]:
from langchain_core.tools import tool

In [9]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [10]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [11]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [12]:
result = multiply.invoke({"a":3, "b":5})
result

15

In [13]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [14]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 - Using StructuredTool

In [15]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [18]:
class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to add")
    b: int = Field(description="The second number to add")

In [19]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [20]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [21]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}


## Method 3 - Using BaseTool Class

In [22]:
from langchain.tools import BaseTool
from typing import Type

In [23]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to add")
    b: int = Field(description="The second number to add")

In [24]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [25]:
multiply_tool = MultiplyTool()

In [26]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}


## Toolkit

In [27]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [28]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [29]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
